1. Need to go through each on and convert hits for instruments to lists
2. Probably easiest to make sure dataframe length is divisible by 16, if not, delete the number from the tail to make that happen
e.g. In UC321, the last row is "SD 1", so we can remove this
3. Create HVO objects for 32 hits
4. For velocities, we are using the "* Peak" columns. I've noticed some values above 1, so we need to normalize these. However, there are many values that are low, e.g. 0.004, 0.05, etc... How do we manage these? -> just divide by max peak (maybe for column), just to keep things at 1 or below
5. Recordings UC_311, UC_312, UC_321 contain four drummers, the others only have three (UC_211 for example). How do we convert these to four streams? -> just keep as is i.e R2 is just zeros


In [27]:
import csv
import os
import note_seq
import numpy as np
import pretty_midi
import pandas as pd

In [28]:
df = pd.read_csv('Annotations/UC_321/UC_321_Onsets_Selected.csv')

In [29]:
print(df.tail(1))

      Take  N   Label SD  SD   C           P          R1          R2  R1 part  \
1728   321  4  109:01:00   1 NaN  198.082059  198.075584  198.064658      NaN   

      R2 part  C Dens  P Dens  R1 Dens  R2 Dens  C Peak  P Peak  R1 Peak  \
1728      NaN     NaN     6.0      7.5     12.0     NaN     NaN      NaN   

      R2 Peak  
1728      NaN  


In [30]:
print(df.shape)

(1729, 18)


In [73]:
print(df['R1 Peak'].max())

1.002148


In [31]:
r1_list = df['R1'].tolist()
r2_list = df['R2'].tolist()
c_list = df['C'].tolist()
p_list = df['P'].tolist()

In [32]:
print(r1_list)

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 2.049816, nan, nan, nan, 2.571809, nan, nan, 2.931527, nan, 3.225729, nan, nan, 3.611234, nan, nan, nan, 4.154318, nan, nan, nan, nan, nan, 4.947183, nan, nan, 5.330822, nan, nan, 5.724812, 5.877038, nan, 6.08383, 6.264425, 6.435412, nan, 6.633239, 6.799894, nan, nan, 7.173024, nan, 7.455055, 7.5776, nan, nan, 7.976443, nan, nan, 8.386853, nan, nan, nan, 8.891968, nan, nan, 9.270569, nan, nan, 9.674225, nan, nan, 10.076645, nan, nan, 10.463685, nan, nan, nan, 10.987489, nan, nan, nan, nan, nan, 11.76128, nan, nan, 12.176175, nan, nan, 12.574656, nan, nan, 12.952934, 13.116342, nan, nan, 13.472264, nan, nan, 13.895775, nan, nan, 14.280268, nan, nan, 14.665443, nan, nan, nan, nan, nan, 15.448251, nan, nan, 15.858495, nan, nan, 16.251806, nan, nan, nan, 16.825775, nan, nan, nan, 17.318763, nan, nan, 17.684752, nan, nan, 18.108406, nan, nan, 18.484294, nan, nan, 18.874798, nan, nan, 19.238615, 19.396821, nan, nan, 19.7673, nan, 2

In [33]:
print(len(r1_list))

1729


In [76]:
int(len(df)/32)


54

In [43]:
print(len(r1_list)%16)
print((len(r1_list) - 1)/16)

1
108.0


In [35]:
from hvo_sequence import HVO_Sequence
from hvo_sequence import ROLAND_REDUCED_MAPPING

In [36]:
Four_Voices = {
        "voice_1": [36],
        "voice_2": [37],
        "voice_3": [38],
        "voice_4": [39]
    }

In [37]:
# create an instance of HVO_Sequence
hvo_seq_uc321 = HVO_Sequence(
    beat_division_factors=[4],                 # select the subdivision of the beat
    drum_mapping=Four_Voices
)

In [38]:
# Add time_signature
hvo_seq_uc321.add_time_signature(time_step=0, numerator=4, denominator=4)

# Add tempo
hvo_seq_uc321.add_tempo(time_step=0, qpm=120)

In [67]:
# Create a random hvo for 32 time steps and 9 voices
hvo_seq_uc321.zeros(32)
hvo_seq_uc321.hvo.shape, hvo_seq_uc321.hits.shape, hvo_seq_uc321.hits#, hvo_seq.velocities, hvo_seq.offsets,

((32, 12),
 (32, 4),
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]))

In [ ]:
for i in range(32): #int(len(df)/32)):
    print(df['C'][i])
    if df['C'][i] > 0:
        print("true")
        hvo_seq_uc321.hits[i][0] = 1
    print(df['P'][i])
    if df['P'][i] > 0:
        print("true")
        hvo_seq_uc321.hits[i][1] = 1
    print(df['R1'][i])
    if df['R1'][i] > 0:
        print("true")
        hvo_seq_uc321.hits[i][2] = 1
    print(df['R2'][i])
    if df['R2'][i] > 0:
        print("true")
        hvo_seq_uc321.hits[i][3] = 1

print(hvo_seq_uc321.hits)


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
2.049816
true
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
2.571809
true
nan
2.687743
true
nan
nan
nan
2.808951
true
nan
nan
nan
2.941593
true
nan
2.931527
true
nan
3.092489
true
nan
nan
nan
3.222104
true
nan
3.225729
true
3.203603
true
3.345761
true
nan
nan
nan
3.464537
true
nan
nan
3.44954
true
3.616191
true
nan
3.611234
true
3.6149
true
3.749529
true
nan
nan
nan
3.866008
true
nan
nan
nan
3.992681
true
nan
nan
3.997539
true
4.156921
true
nan
4.154318
true
4.163945
true
4.289168
true
nan
nan
nan
4.403388
true
nan
nan
nan
4.528062
true
nan
nan
4.523231
true
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 

In [ ]:
for i in range(32):
    print(df['C Peak'][i])
    if df['C Peak'][i] > 0:
        print("true")
        hvo_seq_uc321.velocities[i][0] = df['C Peak'][i]
    print(df['P Peak'][i])
    if df['P Peak'][i] > 0:
        print("true")
        hvo_seq_uc321.velocities[i][1] = df['P Peak'][i]
    print(df['R1 Peak'][i])
    if df['R1 Peak'][i] > 0:
        print("true")
        hvo_seq_uc321.velocities[i][2] = df['R1 Peak'][i]
    print(df['R2 Peak'][i])
    if df['R2 Peak'][i] > 0:
        print("true")
        hvo_seq_uc321.velocities[i][3] = df['R2 Peak'][i]

print(hvo_seq_uc321.velocities)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
0.005278
true
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
0.029197
true
nan
0.072584
true
nan
nan
nan
0.070041
true
nan
nan
nan
0.088111
true
nan
0.009988
true
nan
0.307606
true
nan
nan
nan
0.072354
true
nan
0.006942
true
0.00351
true
0.095679
true
nan
nan
nan
0.11665
true
nan
nan
0.027006
true
0.29602
true
nan
0.040458
true
0.020396
true
0.071433
true
nan
nan
nan
0.179092
true
nan
nan
nan
0.158136
true
nan
nan
0.026769
true
0.175568
true
nan
0.020822
true
0.013287
true
0.069193
true
nan
nan
nan
0.168712
true
nan
nan
nan
0.189697
true
nan
nan
0.01981
true
[[ 0.        0.        0.        0.      ]
 [ 0.        0.        0.        0.      ]
 [ 0.        0.        0.        0.      ]
 [ 0.        0.        0.        0.      ]
 [ 0.        0.        0.        0.      ]
 [ 0.  